In [ ]:
#calculate of steam reform of methane

#According to the obtained K value, the amount of each substance reached equilibrium at different temperatures and pressures is calculated
import numpy as np
from scipy.optimize import fsolve
import pandas as pd


#As a function of temperature for three different reactions, different T corresponds to different K
def gibbs_equation(T):
    delta_G1 = 86250 - 107.64 * T
    K1 = np.exp(-delta_G1 / (8.314 * T))
    
    delta_G2 = -33600 + 29.4 * T
    K2 = np.exp(-delta_G2 / (8.314 * T))
    
    delta_G3 = 137100 - 145.06 * T
    K3 = np.exp(-delta_G3 / (8.314 * T))
       
    return K1, K2, K3


#Calculate the amount of each substance at different temperatures, pressures, 
#and when the amount of initial raw material added reaches equilibrium
def calculate_equilibrium_constants(variables, T, P, N_ch4, N_h2o, N_co2, K1, K2, K3):
    x, y, z = variables
    
    p1 = P / 101300

#An expression for calculating the amount of each substance when equilibrium is reached
    E_ch4 = N_ch4 - x
    E_h2o = N_h2o - y-z
    E_h2 = 2*x+y+z
    E_co2 = y
    E_co = z-y
    E_c = x-z
    
    E_total = E_ch4 + E_h2o + E_co2 + E_h2 + E_co 

#The proportion of matter that reaches equilibrium to the total matter
    x_ch4 = E_ch4 / E_total
    x_h2o = E_h2o / E_total
    x_h2 = E_h2 / E_total
    x_co2 = E_co2 / E_total
    x_co = E_co / E_total
    x_c = E_c

#A formula for the quantities of each substance calculated using the equilibrium constant K
    eq1 = (x_h2*x_h2 * p1)  / (x_ch4 )  - K1
    eq2 = (x_h2 *x_co2)  / (x_co*x_h2o )  - K2
    eq3 = (x_co*x_h2 * p1) /  (x_h2o )  - K3

    return np.array([eq1, eq2, eq3])


# Check the constraint function, thus simplifying the programming calculation difficulty.
#According to the calculated values of K at different temperatures,
#give different intervals corresponding to K within a certain deviation range, 
#and check whether the obtained equilibrium state results are within the interval

def check_constraints(E_co, E_c, E_h2, E_h2o, E_ch4, E_co2, x, y, z, N_co2, N_h2o, N_ch4, T, K11, K22, K33):
    return (E_co > 0 
            and E_c > 0 
            and E_h2 > 0 
            and E_h2o > 0 
            and E_ch4 > 0 
            and E_co2 > 0 
            and x > 0 
            #and y > 0
            and z > 0 
            and ((T == 800 and 0.9 <= K11 <= 1.1 and 4.3 <= K22 <= 4.7 and 0.03 <= K33 <= 0.06) or
                 (T == 850 and 2 <= K11 <= 2.3 and 3.1 <= K22 <= 3.6 and 0.12 <= K33 <= 0.16) or
                 (T == 900 and 4 <= K11 <= 4.3 and 2.2 <= K22 <= 2.8 and 0.3 <= K33 <= 0.5) or
                 (T == 950 and 7.3 <= K11 <= 7.8 and 1.9 <= K22 <= 2.2 and 0.9 <= K33 <= 1.2) or
                 (T == 1000 and 12 <= K11 <= 14 and 1.4<= K22 <= 1.8 and 2.4 <= K33 <= 3) or
                 (T == 1050 and 20 <= K11 <= 23 and 1.1<= K22 <= 1.5 and 5.3 <= K33 <= 6) or
                 (T == 1100 and 31 <= K11 <= 35 and 1 <= K22 <= 1.3 and 10 <= K33 <= 13) or
                 (T == 1150 and 50 <= K11 <= 52 and 0.8 <= K22 <= 1.2 and 22 <= K33 <= 24) or
                 (T == 1200 and 70 <= K11 <= 77 and 0.7 <= K22 <= 0.9 and 39 <= K33 <= 43) or
                 (T == 1250 and 100 <= K11 <= 110 and 0.6 <= K22 <= 0.8 and 68 <= K33 <= 72) or
                 (T == 1300 and 135 <= K11 <= 149 and 0.6 <= K22 <= 0.7 and 105 <= K33 <= 129) ))
                 

# Create an empty DataFrame to store the results
columns = ['T', 'P', 'N_ch4', 'N_h2o', 'N_co2', 'x', 'y', 'z', 'E_ch4', 'E_h2o', 'E_co2', 'E_h2', 'E_co', 'E_c', 'E_total']
results_df = pd.DataFrame(columns=columns)

#  Generate N_h2o_range and N_co2_range.
#That is, the amount of water and carbon dioxide initially added to control the feed ratio. 
#You can set different values according to your needs.
N_h2o_range = [i * 0.1 for i in range(6, 10)]
N_co2_range = [i * 0 for i in range(0, 1)]

#At different temperature and pressure ranges, the amount of fixed methane added is 1
T_range = range(800, 1301, 100)
P_range = range(100000, 1000001, 100000)
N_ch4_range = range(1, 2, 1)

# We're going to do a for loop for all of them
for T in T_range:
    for P in P_range:
        for N_ch4 in N_ch4_range:
            for N_h2o in N_h2o_range:
                for N_co2 in N_co2_range:
                    # Flag indicating whether a result has been found
                    found_result = False

                   
# For each set of values T, P, N_ch4, N_h2o, N_co2, the results of all possible initial guesses are calculated
#Self-set results need to be preliminarily estimated based on the amount of additions, thus reducing calculation time
                    for x_guess in np.arange(0, 1.01, 0.01):
                        for y_guess in np.arange(-0.51, 0.51, 0.01):
                            for z_guess in np.arange(0, 1.01, 0.01):
                                initial_guess = [x_guess, y_guess, z_guess]
                                K1, K2, K3 = gibbs_equation(T)
                                result = fsolve(calculate_equilibrium_constants, initial_guess, args=(T, P, N_ch4, N_h2o, N_co2, K1, K2, K3), xtol=1e-7, maxfev=100000)
                                
                                # Process the result and add it to the DataFrame
                                #x, y, z = np.around(result, 4)  
                                # Keep four decimal places, which is to consider the size of the k value, 
                                #but also the data is too small, so take the significant number
                                x = np.around(result[0], 9)  
                                y = np.around(result[1], 9)  
                                z = np.around(result[2], 9)  
                                p1 = P / 101300
                                
#The results were processed, the final distribution proportion of each group was obtained, and the equilibrium constant K was calculated
                                E_ch4 = N_ch4 - x
                                E_h2o = N_h2o - y-z
                                E_h2 = 2*x+y+z
                                E_co2 = y
                                E_co = z-y
                                E_c = x-z
                                E_total = E_ch4 + E_h2o + E_co2 + E_h2 + E_co 
    


                                x_ch4 = E_ch4 / E_total
                                x_h2o = E_h2o / E_total
                                x_h2 = E_h2 / E_total
                                x_co2 = E_co2 / E_total
                                x_co = E_co / E_total
                                x_c = E_c
    
                                K11 = (x_h2 *x_h2* p1)  / (x_ch4 )  
                                K22 = (x_h2 *x_co2)  / (x_co*x_h2o ) 
                                K33 = (x_co*x_h2 * p1) /  (x_h2o )  
                                
                                # Add restriction
                                if check_constraints(E_co, E_c, E_h2, E_h2o, E_ch4, E_co2, x, y, z, N_co2, N_h2o, N_ch4, T, K11, K22, K33):
                                    # output result
                                    print(f'T: {T}, P: {P}, N_ch4: {N_ch4}, N_h2o: {N_h2o}, N_co2: {N_co2}, x: {x}, y: {y}, z: {z}, E_total: {E_total}')
                                    print(f'E_ch4: {E_ch4}, E_h2o: {E_h2o}, E_co2: {E_co2}, E_h2: {E_h2}, E_co: {E_co}, E_c: {E_c}')


                                    # Add the result to the DataFrame
                                    new_row = pd.DataFrame([{
                                               'T': T, 'P': P, 'N_ch4': N_ch4, 'N_h2o': N_h2o, 'N_co2': N_co2,
                                               'x': x, 'y': y, 'z': z,
                                               'E_ch4': E_ch4, 'E_h2o': E_h2o, 'E_co2': E_co2, 'E_h2': E_h2, 'E_co': E_co, 'E_c': E_c, 'E_total': E_total
                                                 }])
                                    results_df = pd.concat([results_df, new_row], ignore_index=True)

                                    
                                    # Set the flag that a result was found
                                    found_result = True
                                    
                                    # After finding the result, the calculation of other initial guesses is stopped, 
                                    #and the next cycle of T, P, N_ch4, N_h2o, N_co2 is carried out
                                    break
                                    
                                # If the result has been found, then the inner loop is directly removed
                                if found_result:
                                    break
                            if found_result:
                                break
                        if found_result:
                            break
                    if found_result:
                        break

# Save the DataFrame as an Excel file
results_df.to_excel('output_CH4_H2O.xlsx', index=False)